In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [19]:
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

In [12]:
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-4o-mini", max_tokens=200)

### Carregar Arquivo PDF

In [13]:
pdf = "lei_ia_2023.pdf"
loader = PyPDFLoader(pdf, extract_images=False)
pages = loader.load_and_split()

### Chunks

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function = len,
    add_start_index = True
)

chuncks = text_splitter.split_documents(pages)

### Salvando Chunks no VectorDB

In [15]:
db = Chroma.from_documents(chuncks, embedding=embeddings_model, persist_directory="chroma_db")
db.persist()

C:\Users\Maria Fernanda\AppData\Local\Temp\ipykernel_20436\2653934535.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


### Método Retriver e Chain

In [16]:
vector_db = Chroma(persist_directory="chroma_db", embedding_function=embeddings_model)

# Load the retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 5})

# Construct the QA chain for the chatbot
chain = load_qa_chain(llm, chain_type="stuff")

### Execução e Query

In [17]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True)) ['output_text']
    return answer

In [18]:
user_question = input("Ask me anything: ")
answer = ask(user_question)
print("Answer:", answer)

C:\Users\Maria Fernanda\AppData\Local\Temp\ipykernel_20436\310251223.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True)) ['output_text']


Answer: Os principais pontos que sua empresa deve se preocupar em relação à lei sobre inteligência artificial incluem:

1. **Ambiente Regulatório Experimental (Sandbox Regulatório)**: Sua empresa pode solicitar autorização para operar em um ambiente regulatório experimental, mas deve apresentar um projeto que demonstre inovação e benefícios sociais.

2. **Direitos dos Afetados**: É necessário garantir que as pessoas afetadas por sistemas de inteligência artificial recebam informações claras sobre como esses sistemas funcionam, incluindo a transparência sobre o uso de dados pessoais.

3. **Sanções e Responsabilidades**: A empresa pode enfrentar sanções administrativas, civis ou penais em caso de infrações, incluindo multas e suspensão de atividades. Além disso, há a obrigação de reparação integral do dano causado.

4. **Monitoramento e Comunicação de Incidentes**: A empresa deve implementar mecanismos de monitoramento contínuo e reportar à autoridade competente qualquer incidente grave 